In [1]:
import pandas as pd
from collections import defaultdict

In [18]:
platform = 'ig'
# platform = 'fb'

df = pd.read_json(f'data/{platform}/{"log20240614050853" if platform=="ig" else "log20240614080312"}.njson', lines=True)

In [19]:
len(df)

2300

In [20]:
df.head()

,platformId,platform,date,updated,type,description,postUrl,subscriberCount,score,media,statistics,account,likeAndViewCountsDisabled,history,languageCode,legacyId,id,scraped,imageText,expandedLinks
0,3.389795e+28,Instagram,2024-06-13 21:45:28,2024-06-14 04:45:35,photo,Scusate il ritardo: sono stati giorni convulsi...,https://www.instagram.com/p/C8K-P_OsCwA/,76688,-1.215128,"[{'type': 'photo', 'url': 'https://scontent-se...","{'actual': {'favoriteCount': 1988, 'commentCou...","{'id': 23300608, 'name': 'Cecilia Strada', 'ha...",False,"[{'actual': {'favoriteCount': 1988, 'commentCo...",it,0,23300608|3389795447117720576,2024-06-14 09:08:56,NaN,NaN
1,3.389730e+28,Instagram,2024-06-13 19:35:22,2024-06-14 05:48:50,photo,Quello che sta succedendo nel Paese e nelle su...,https://www.instagram.com/p/C8KvXBctwDk/,6539,-6.714286,"[{'type': 'photo', 'url': 'https://scontent-se...","{'actual': {'favoriteCount': 0, 'commentCount'...","{'id': 20152676, 'name': 'Enzo Maraio', 'handl...",True,"[{'actual': {'favoriteCount': 0, 'commentCount...",it,0,20152676|3389729959839203556,2024-06-14 09:08:56,Enzo Maraio Quello che sta succedendo nel Paes...,NaN
2,3.389726e+28,Instagram,2024-06-13 19:27:17,2024-06-14 06:13:31,photo,“Bella Ciao” e’ canto simbolo della Resistenza...,https://www.instagram.com/p/C8Kub0HtsvQ/,206940,3.717406,"[{'type': 'photo', 'url': 'https://scontent-se...","{'actual': {'favoriteCount': 4957, 'commentCou...","{'id': 2870271, 'name': 'Stefano Bonaccini', '...",False,"[{'actual': {'favoriteCount': 4957, 'commentCo...",it,0,2870271|3389725891079097296,2024-06-14 09:08:57,PRIMA GLI ITALIANI A,NaN
3,3.389726e+28,Instagram,2024-06-13 19:27:01,2024-06-14 06:33:32,photo,Le sanzioni stabilite dall’Ufficio di presiden...,https://www.instagram.com/p/C8KuZ7dszQx/,1788722,2.459154,"[{'type': 'photo', 'url': 'https://scontent-se...","{'actual': {'favoriteCount': 15026, 'commentCo...","{'id': 3445041, 'name': 'Giuseppe Conte', 'han...",False,"[{'actual': {'favoriteCount': 15026, 'commentC...",it,0,3445041|3389725761525199921,2024-06-14 09:08:57,NaN,NaN
4,3.389704e+27,Instagram,2024-06-13 18:42:49,2024-06-14 07:09:39,album,"In piazza contro ogni autonomia differenziata,...",https://www.instagram.com/p/C8KpWJxN3A6/,76978,1.410909,"[{'type': 'photo', 'url': 'https://scontent-se...","{'actual': {'favoriteCount': 1125, 'commentCou...","{'id': 12106013, 'name': 'Nicola Fratoianni', ...",False,"[{'actual': {'favoriteCount': 1125, 'commentCo...",it,0,12106013|3389703511774556218,2024-06-14 09:08:58,NaN,NaN


In [21]:
mask = (df['date'] >= '2024-5-13') & (df['date'] <= '2024-6-10')
df = df.loc[mask]
len(df)

2144

In [22]:
histories = defaultdict(list)

def add_to_histories(row):
    post_id = row.id
    post_date = row.date
    for entry in row.history:
        histories[post_id].append(pd.Series(entry['actual']|{'history_date':entry['date']}|{'timestep':entry['timestep']}|{'post_date':post_date}))
_=df.apply(add_to_histories, axis=1)

In [23]:
for k, v in histories.items():
    print(v)
    break

[favoriteCount                   8684
commentCount                     425
history_date     2024-06-14 00:31:37
timestep                          55
post_date        2024-06-09 23:06:19
dtype: object, favoriteCount                   8683
commentCount                     425
history_date     2024-06-13 17:49:02
timestep                          54
post_date        2024-06-09 23:06:19
dtype: object, favoriteCount                   8675
commentCount                     420
history_date     2024-06-13 10:47:24
timestep                          52
post_date        2024-06-09 23:06:19
dtype: object, favoriteCount                   8669
commentCount                     417
history_date     2024-06-12 22:40:56
timestep                          50
post_date        2024-06-09 23:06:19
dtype: object, favoriteCount                   8653
commentCount                     405
history_date     2024-06-12 14:56:40
timestep                          49
post_date        2024-06-09 23:06:19
dtype: object,

In [65]:
# documentation for timesteps: https://help.crowdtangle.com/en/articles/3213537-crowdtangle-codebook
def process_timesteps(k, v):
    vv = pd.DataFrame(v)
    def _process(history_24, suffix):
        if platform=='ig':
            history_24 = history_24[['commentCount', 'favoriteCount']].rename(columns=lambda x:x+suffix)
        else:
            history_24['reactionCount'] = history_24[['likeCount', 'loveCount', 'wowCount',
                   'hahaCount', 'sadCount', 'angryCount', 'thankfulCount', 'careCount',
                   ]].sum(axis=1)
            history_24 = history_24[['likeCount', 'reactionCount', 'commentCount', 'shareCount']].rename(columns=lambda x:x+suffix)
        history_24['post_id'] = k
        history_24.set_index('post_id', inplace=True)
        return history_24

    history_24 = vv[vv.timestep.isin([38-1, 38, 38+1])].head(2).tail(1).copy()
    history_24=_process(history_24, '24')

    history_48 = vv[vv.timestep.isin([46-1, 46, 46+1])].head(2).tail(1).copy()
    history_48=_process(history_48, '48')

    return pd.concat((history_24, history_48), axis=1)


In [66]:
derived_measures = list()
for k, v in histories.items():
    derived_measures.append(process_timesteps(k, v))

In [67]:
derived_measures_df = pd.concat(derived_measures)

In [68]:
derived_measures_df.head()

,commentCount24,favoriteCount24,commentCount48,favoriteCount48
post_id,,,,
16834706|3386937034207327337,337.0,8425.0,399.0,8616.0
20152662|3386937034207327337,339.0,8469.0,390.0,8603.0
2060021|3386931859680649189,429.0,12884.0,440.0,13120.0
3719710|3386931859680649189,423.0,12827.0,439.0,13141.0
12106013|3386928469597852663,1055.0,24790.0,983.0,25379.0


In [69]:
merged_df = pd.merge(df, derived_measures_df, left_on='id', right_index=True, how='left')

In [70]:
merged_df.head()

,platformId,platform,date,updated,type,description,postUrl,subscriberCount,score,media,...,languageCode,legacyId,id,scraped,imageText,expandedLinks,commentCount24,favoriteCount24,commentCount48,favoriteCount48
156,3.386937e+29,Instagram,2024-06-09 23:06:19,2024-06-14 00:31:37,photo,Un risultato straordinario!\n\nStiamo costruen...,https://www.instagram.com/p/C8A0UmwivRp/,39271,32.532143,"[{'type': 'photo', 'url': 'https://scontent-se...",...,it,0,16834706|3386937034207327337,2024-06-14 09:10:16,IL ILCORAGGIO DIOSARE EUROPEE 2024 Alleanza Ve...,NaN,337.0,8425.0,399.0,8616.0
157,3.386937e+29,Instagram,2024-06-09 23:06:19,2024-06-14 04:30:43,photo,Un risultato straordinario!\n\nStiamo costruen...,https://www.instagram.com/p/C8A0UmwivRp/,5983,49.505435,"[{'type': 'photo', 'url': 'https://scontent-se...",...,it,0,20152662|3386937034207327337,2024-06-14 09:10:17,IL ILCORAGGIO DIOSARE EUROPEE 2024 Alleanza Ve...,NaN,339.0,8469.0,390.0,8603.0
158,3.386932e+28,Instagram,2024-06-09 22:56:02,2024-06-14 05:45:58,photo,"Un risultato straordinario, #ForzaItalia cresc...",https://www.instagram.com/p/C8AzJTmt9_l/,91180,13.188751,"[{'type': 'photo', 'url': 'https://scontent-se...",...,it,0,2060021|3386931859680649189,2024-06-14 09:10:17,SPBZO FORZA SILVO,NaN,429.0,12884.0,440.0,13120.0
159,3.386932e+28,Instagram,2024-06-09 22:56:02,2024-06-14 08:06:35,photo,"Un risultato straordinario, #ForzaItalia cresc...",https://www.instagram.com/p/C8AzJTmt9_l/,58211,13.730159,"[{'type': 'photo', 'url': 'https://scontent-se...",...,it,0,3719710|3386931859680649189,2024-06-14 09:10:18,SPBZO FORZA SILVO,NaN,423.0,12827.0,439.0,13141.0
160,3.386928e+27,Instagram,2024-06-09 22:49:18,2024-06-14 07:09:39,photo,Avevamo promesso che saremmo stati la vera sor...,https://www.instagram.com/p/C8AyX-WCZP3/,73236,20.299923,"[{'type': 'photo', 'url': 'https://scontent-se...",...,it,0,12106013|3386928469597852663,2024-06-14 09:10:18,DI CORAGGIO EUROPEE EUROPEE2024 2024 OSARE All...,NaN,1055.0,24790.0,983.0,25379.0


In [71]:
merged_df.to_csv(f'data/{platform}_merged.csv', index=False)

In [72]:
derived_measures_df.to_csv(f'data/{platform}_engagement.csv', index=True)

In [73]:
# dal 13/5--10/6 dopo 24 e 48h

# like_24h | like_48 | total_engagement_24h | total_engagement_48h | share_24h | share_48 | comment_24h | comment_48

In [76]:
sorted(merged_df[merged_df.commentCount24.isna()].iloc[0].history, key=lambda x: x['timestep'])

[{'actual': {'favoriteCount': 224, 'commentCount': 0},
  'expected': {'favoriteCount': 44, 'commentCount': 2},
  'timestep': 13,
  'date': '2024-06-10 01:09:37',
  'score': 4.869565217391305},
 {'actual': {'favoriteCount': 244, 'commentCount': 1},
  'expected': {'favoriteCount': 49, 'commentCount': 2},
  'timestep': 24,
  'date': '2024-06-10 06:20:05',
  'score': 4.803921568627451},
 {'actual': {'favoriteCount': 255, 'commentCount': 1},
  'expected': {'favoriteCount': 54, 'commentCount': 2},
  'timestep': 27,
  'date': '2024-06-10 07:51:57',
  'score': 4.571428571428571},
 {'actual': {'favoriteCount': 278, 'commentCount': 1},
  'expected': {'favoriteCount': 54, 'commentCount': 2},
  'timestep': 30,
  'date': '2024-06-10 11:00:05',
  'score': 4.982142857142857},
 {'actual': {'favoriteCount': 295, 'commentCount': 2},
  'expected': {'favoriteCount': 54, 'commentCount': 2},
  'timestep': 36,
  'date': '2024-06-10 16:48:08',
  'score': 5.303571428571429},
 {'actual': {'favoriteCount': 312, 

In [77]:
merged_df[merged_df.commentCount24.isna()].history.apply(len)

165     19
167     20
188     18
191     19
196     22
        ..
2281    34
2291    21
2294    25
2295    20
2297    18
Name: history, Length: 484, dtype: int64